In this notebook, we will create and compare several models. Our target feature for this notebooks is (Likes+Comments/(Views+1))
which we will call "engagement". We have added 1 to the denonimator just to ensure we never divide by zero. 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"all_features_final.csv")
features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref", "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products", "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]

#Create the target column $y$ here 

df["y"] = (df["likes"] + df["commentsCount"])  / (df["viewCount"] + 1) 

#get rid of noisy columns
df = df[ features + ["y"] ] 

In [2]:
#import everything

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split


In [3]:
#Do an 80-20 Train Test Split Here. Never ever touch the testing set please!

df_train, df_test = train_test_split(df, shuffle = True, test_size = .2) #We can't stratify because we have too many categorical features. I hope this is ok
#DO NOT TOUCH THE ABOVE X_TEST VARIABLE FOR ANY REASON

#We want a very basic idea of the MSE for each model, before we do proper cross-validation. We use a secondary split for this.
df_tt, df_ho = train_test_split(df_train, shuffle = True, test_size = .2)


In [4]:
#Create the baseline model here

In [ ]:
#Creat the basic linear regression model here 

###Fitting the basic linear regression model using the training set and print the summary of the model.
model = LinearRegression()
model.fit(df_tt[features], df_tt["y"])
# Evaluate the model
y_pred = model.predict(df_ho[features])
rmse = root_mean_squared_error(df_ho["y"], y_pred)
r2 = r2_score(df_ho["y"], y_pred)
print(f"Root Mean Squared Error: {rmse:.6f}")
print(f"R-squared: {r2:.4f}")

In [6]:
#Create the basic linear regression model here with lasso regression. 

In [7]:
#Create a model whose features include all interaction terms
pipe = Pipeline([  ("interaction terms", PolynomialFeatures(degree = 2, interaction_only = True, include_bias = False) ),
                   ("linear model", LinearRegression())
]) 
#setting degree = 2 creates all pairwise interaction terms. 

pipe.fit( df_tt[features], df_tt["y"]) 
pred = pipe.predict( df_ho[features] )

print( root_mean_squared_error( df_ho["y"], pred) ) 

0.03158406604410635


In [14]:
#Create a model with all interaction terms and lasso regression 
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_tt[features] = scaler.fit_transform( df_tt[features] )
df_ho[features] = scaler.transform( df_ho[features] )

# using lasso cv to find the best alpha
# lasso = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=np.logspace(-4, 1, 30))
# lasso.fit(df_tt[features], df_tt["y"])
# pred = lasso.predict(df_ho[features])
# print("Lasso CV MSE:", root_mean_squared_error(df_ho["y"], pred))
# print("Optimal alpha:", lasso.alpha_)

pipe = Pipeline([
    ("interaction terms", PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
    ("lasso", Lasso(alpha=0.0001, max_iter=10000))
])
pipe.fit(df_tt[features], df_tt["y"])
pred = pipe.predict(df_ho[features])
print("MSE : ", root_mean_squared_error(df_ho["y"], pred))

# Get lasso coefficients
lasso_coeffs = pd.Series(pipe.named_steps['lasso'].coef_, index=pipe.named_steps['interaction terms'].get_feature_names_out(features))
lasso_coeffs = lasso_coeffs[lasso_coeffs != 0]
# print(lasso_coeffs)

MSE :  0.031542021171636604


In [9]:
#Do cross-validation to compare all models 

In [10]:
#Final interpretation 